## Problema 2:

### Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides  apresentado no trabalho TP3

 a) Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função  $$\gcd$$  é   $$\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$$ .

In [ ]:
from z3 import *

def verifica_pos_condicao_z3(a_val, b_val, r_val, s_val, t_val):
    # Criar solver
    solver = Solver()
    
    # Declarar variáveis
    a = Int('a')
    b = Int('b')
    r = Int('r')
    s = Int('s')
    t = Int('t')
    
    # Adicionar valores concretos
    solver.add(a == a_val)
    solver.add(b == b_val)
    solver.add(r == r_val)
    solver.add(s == s_val)
    solver.add(t == t_val)
    
    # Pós-condição
    pos_cond = And(
        r > 0,                    # r é positivo
        r == a * s + b * t,      # r é combinação linear de a e b
        ForAll([Int('x'), Int('y')],  # Para todo x,y
            Implies(
                And(
                    Int('x') == a * Int('y') + b * Int('y'),
                    Int('x') > 0
                ),
                Int('x') >= r     # r é o menor valor positivo
            )
        )
    )
    
    solver.add(pos_cond)
    
    # Verificar
    if solver.check() == sat:
        modelo = solver.model()
        return True, f"Pós-condição satisfeita: {r_val} = {a_val}*{s_val} + {b_val}*{t_val}"
    else:
        return False, "Pós-condição não satisfeita"

def testar_pos_condicao_z3():
    casos_teste = [
        (48, 36, 12, 1, -1),    # Caso válido
        (17, 5, 1, -2, 7),      # Caso válido
        (48, 36, 12, -1, 2)     # Caso inválido
    ]
    
    for a, b, r, s, t in casos_teste:
        print(f"\nTestando a={a}, b={b}, r={r}, s={s}, t={t}")
        sucesso, mensagem = verifica_pos_condicao_z3(a, b, r, s, t)
        print(f"Resultado: {'Sucesso' if sucesso else 'Falha'}")
        print(f"Mensagem: {mensagem}")

if __name__ == "__main__":
    print("Testando Pós-condição com Z3:")
    testar_pos_condicao_z3()


Testando Pós-condição com Z3:

Testando a=48, b=36, r=12, s=1, t=-1
Resultado: Sucesso
Mensagem: Pós-condição satisfeita: 12 = 48*1 + 36*-1

Testando a=17, b=5, r=1, s=-2, t=7
Resultado: Sucesso
Mensagem: Pós-condição satisfeita: 1 = 17*-2 + 5*7

Testando a=48, b=36, r=12, s=-1, t=2
Resultado: Falha
Mensagem: Pós-condição não satisfeita


 b) Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .

In [ ]:
from z3 import *

def programa_lpa_z3(a_val, b_val):
    solver = Solver()
    
    # Criar variáveis simbólicas para o estado inicial
    a = Int('a')
    b = Int('b')
    r = Int('r')
    r_linha = Int('r_linha')
    s = Int('s')
    s_linha = Int('s_linha')
    t = Int('t')
    t_linha = Int('t_linha')
    q = Int('q')
    
    # Estado inicial
    solver.add(a == a_val)
    solver.add(b == b_val)
    solver.add(r == a_val)
    solver.add(r_linha == b_val)
    solver.add(s == 1)
    solver.add(s_linha == 0)
    solver.add(t == 0)
    solver.add(t_linha == 1)
    
    # Invariante inicial
    solver.add(r == a * s + b * t)
    solver.add(r_linha == a * s_linha + b * t_linha)
    
    estados = []
    r_atual, r_linha_atual = a_val, b_val
    s_atual, s_linha_atual = 1, 0
    t_atual, t_linha_atual = 0, 1
    
    while r_linha_atual != 0:
        q_atual = r_atual // r_linha_atual
        
        # Criar novas variáveis para o próximo estado
        r_prox = Int(f'r_{len(estados)}')
        r_linha_prox = Int(f'r_linha_{len(estados)}')
        s_prox = Int(f's_{len(estados)}')
        s_linha_prox = Int(f's_linha_{len(estados)}')
        t_prox = Int(f't_{len(estados)}')
        t_linha_prox = Int(f't_linha_{len(estados)}')
        
        # Adicionar restrições de transição
        solver.add(r_prox == r_linha_atual)
        solver.add(r_linha_prox == r_atual - q_atual * r_linha_atual)
        solver.add(s_prox == s_linha_atual)
        solver.add(s_linha_prox == s_atual - q_atual * s_linha_atual)
        solver.add(t_prox == t_linha_atual)
        solver.add(t_linha_prox == t_atual - q_atual * t_linha_atual)
        
        # Atualizar valores atuais
        r_atual, r_linha_atual = r_linha_atual, r_atual - q_atual * r_linha_atual
        s_atual, s_linha_atual = s_linha_atual, s_atual - q_atual * s_linha_atual
        t_atual, t_linha_atual = t_linha_atual, t_atual - q_atual * t_linha_atual
        
        estados.append((r_atual, r_linha_atual, s_atual, s_linha_atual, t_atual, t_linha_atual, q_atual))
    
    # Verificar se o modelo é satisfatível
    if solver.check() == sat:
        return True, estados
    else:
        return False, "Não foi possível encontrar uma solução válida"

def testar_programa_lpa_z3():
    casos_teste = [
        (48, 36),
        (17, 5),
        (21, 14)
    ]
    
    for a, b in casos_teste:
        print(f"\nExecutando programa LPA para a={a}, b={b}")
        sucesso, resultado = programa_lpa_z3(a, b)
        
        if sucesso:
            print("Estados gerados:")
            for i, estado in enumerate(resultado):
                r, r_linha, s, s_linha, t, t_linha, q = estado
                print(f"Estado {i}: r={r}, r'={r_linha}, s={s}, s'={s_linha}, t={t}, t'={t_linha}, q={q}")
            print(f"MDC encontrado: {resultado[-1][0]}")
        else:
            print(f"Erro: {resultado}")

if __name__ == "__main__":
    print("Testando Programa LPA com Z3:")
    testar_programa_lpa_z3()


Testando Pós-condição com Z3:

Testando a=48, b=36, r=12, s=1, t=-1
Resultado: Sucesso
Mensagem: Pós-condição satisfeita: 12 = 48*1 + 36*-1

Testando a=17, b=5, r=1, s=-2, t=7
Resultado: Sucesso
Mensagem: Pós-condição satisfeita: 1 = 17*-2 + 5*7

Testando a=48, b=36, r=12, s=-1, t=2
Resultado: Falha
Mensagem: Pós-condição não satisfeita


c) Construa codificações do programa LPA através de transformadores de predicados “strongest post-condition” e  prove a correção  do programa LPA.